In [3]:
import os
import datetime
import polars as pl
import pandas as pd
pl.Config(tbl_rows=100)
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.zjisti_vazbu import zjisti_vazbu

In [4]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
705240


In [5]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

363993

In [6]:
vek = pl.read_parquet(os.path.join("data","narozeni-umrti-gender.parquet"))
df = df.join(vek, on="100_7", how="left").with_columns((pl.col("rok") - pl.col("narozeni")).alias("vek"))

In [7]:
df = df.filter((pl.col('umrti') > pl.col('rok')) | (pl.col('umrti').is_null()))

## Budeme brát do úvahy pouze autorstvo vícenásobné!

In [9]:
vicenasobni = df.group_by('100_7').len().filter(pl.col('len') >= 3).select(pl.col('100_7')).to_series().to_list()
len(vicenasobni)

50186

In [10]:
df = df.filter(pl.col('100_7').is_in(vicenasobni))

In [11]:
len(df)

458021

## Kolik % autorstva ten rok skončilo

In [13]:
df.sample(20)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,narozeni,umrti,gender,vek
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,i64,i64,str,i64
"""1""","""Kozák, Jan,""","""jk01062553""","[""aut""]","""1860-1934""",null,null,null,null,"""cpk20233577569""",""" nam a22 i 4500""","""231208s1920 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Druhá početnice pro školy obec…","""obor čísel 1-100 /""","""sepsali Jan Kozák a Jan Roček …",null,null,null,null,null,"[""116 stran :""]","[""ilustrace ;""]","[""22 cm""]",null,null,null,…,null,null,null,null,"[""1"", ""1""]","[""Roček, Jan,"", ""Krátký, Antonín,""]","[""aut"", ""ill""]","[""1858-1925"", ""1882-1937""]","[""jk01102542"", ""jk01062928""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1920,116,null,1860,1934,"""m""",60
"""1""","""Venclík, Vlastimil,""","""jk01142030""","[""aut""]","""1942-""",null,null,null,null,"""ck9101576""",""" nam a22 4500""","""910411s1990 xr u0…","""(brož.) :""","[""10 Kčs""]","[""80-203-0005-8""]",null,null,null,null,null,null,"""1""","""0""","""Nový způsob starého humanismu""","""Komedie o tragédii : Přísně ut…","""Vlastimil Venclík""",null,null,null,null,null,"[""90 s. ;""]",null,"[""29 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1990,90,"""brožovaná""",1942,null,"""m""",48
"""1""","""Herfort, Karel,""","""jk01040816""","[""aut""]","""1871-1940""",null,null,null,null,"""bk193101099""",""" nam a22 1 4500""","""990826s1931 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Rodokmeny slabomyslných""","""Pedegrees of feeble-minded : S…","""Univ. prof. MUDr. Karel Herfor…","[""[IV. díl] ="", ""[IV. part] /""]",null,null,null,null,"[""46 s. ;""]",null,"[""4°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1931,46,null,1871,1940,"""m""",60
"""1""","""Kašpárková, Věra""","""jx20050727055""","[""aut""]",null,null,null,null,null,"""bk197200534""",""" nam a22 1 4500""","""970114s1971 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Korektivní fonetický kurs angl…","""Určeno pro posl. fak. společ. …","""Autoři: Věra Kašpárková, Helmu…",null,null,null,null,null,"[""43 s. ;""]",null,"[""4°""]",null,null,null,…,null,null,null,null,"[""1""]","[""Kašpárek, Helmut""]","[""aut""]",null,"[""jx20051207026""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1971,43,null,null,null,null,null
"""1""","""Drnek, Jan,""","""xx0004343""","[""aut""]","""1960-""",null,null,null,null,"""nkc20183002307""",""" nam a22 ia4500""","""180615s2018 xr abcee 0…","""(vázáno)""",null,"[""978-80-904746-4-2""]",null,null,null,null,null,null,"""1""","""0""","""Za Malickou branou""","""příběh a historie prvního obvo…","""Jan Drnek, Oto Brachtel""",null,null,null,null,null,"[""2 svazky (832 stran) :""]","[""ilustrace (převážně

In [14]:
df = df.join(df.group_by("100_a").agg(pl.col("rok").max().alias("posledni_kniha")), on="100_a", how="left")

In [15]:
df.filter(pl.col('rok') == pl.col('posledni_kniha'))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,narozeni,umrti,gender,vek,posledni_kniha
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,i64,i64,str,i64,i64
"""1""","""Nezval, Pavel,""","""jo20000080492""","[""aut""]","""1948-""",null,null,null,null,"""cpe20142633681""",""" cam a22 a 4500""","""141017s2014 xr a fs 0…",null,null,"[""978-80-7248-902-2""]",null,null,null,null,null,null,"""1""","""0""","""Rovnováha ekonomiky, zahraničn…","""vědecká monografie /""","""Pavel Nezval, Ingrid Majerová""",null,null,"""[elektronický zdroj] :""",null,null,"[""1 CD-ROM :""]","[""barev. ;""]","[""12 cm""]",null,null,null,…,null,null,null,"[""1""]","[""Majerová, Ingrid,""]","[""aut""]","[""1971-""]","[""mzk2005289906""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2014,1,null,1948,null,"""m""",66,2014
"""1""","""Dokoupilová, Milena""","""jx20070511013""","[""aut""]",null,null,null,null,null,"""nkc20243603453""",""" nam a22 i 4500""","""240530s2024 xr a f 0…","""(vázáno) :""","[""Kč 599,00""]","[""978-80-88506-24-9""]",null,null,null,null,null,null,"""1""","""0""","""Kazuistiky z neonatologie II""",null,"""Milena Dokoupilová, Miloš Čern…",null,null,null,null,null,"[""364 stran :""]","[""ilustrace (převážně barevné) ;""]","[""24 cm""]",null,null,null,…,null,null,null,"[""1""]","[""Černý, Miloš,""]","[""aut""]","[""1957-""]","[""xx0145287""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2024,364,"""pevná""",null,null,null,null,2024
"""1""","""Tan, Amy,""","""xx0008746""","[""aut""]","""1952-""",null,null,null,null,"""nkc20162798851""",""" nam a22 i 4500""","""160517s2016 xr g 0…","""(vázáno)""",null,"[""978-80-267-0615-1""]",null,null,null,null,null,null,"""1""","""0""","""Klub radosti a štěstí""",null,"""Amy Tan ; přeložila Vladimíra …",null,null,null,null,null,"[""308 stran ;""]",null,"[""21 cm""]",null,null,null,…,null,null,null,"[""1""]","[""Šefranka Žáková, Vladimíra,""]","[""trl""]","[""1961-""]","[""jx20050707064""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2016,308,"""pevná""",1952,null,"""f""",64,2016
"""1""","""Gogol‘, Nikolaj Vasil‘jevič,""","""jn19990210244""","[""aut""]","""1809-1852""",null,null,null,null,"""bknjas05108""",""" nam a22 1 4500""","""011225s1847 xr g |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Něvský prospekt""",null,"""[Nikolaj Vasil'jevič Gogol' ; …",null,null,null,null,null,"[""s. 414-466 ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1847,466,null,1809,1852,"""m""",38,1847
"""1""","""Simons, Sissy,""","""mzk2002103051""","[""aut""]","""1974-""",null,null,null,null,"""nkc19981167627""",""" nam a22 i 4500""","""170802s1997 xr ach e 0…","""(v knize neuvedeno ;""",null,"[""80-238-1910-0""]",null,null,null,null,null,null,"""1""","""0""","""Vzpomínky a jiné oplzlosti""",null,"""Sissy Simons""",null,null,null,null,null,"[""208 stran :""]","[""ilu

In [16]:
podil_koncicich = df.filter(pl.col('100_7').is_in(cesi)).filter(pl.col('rok') == pl.col('posledni_kniha')).group_by('rok').agg(pl.col('100_7').n_unique()).rename({'100_7':'skoncilo'}).join(df.filter(pl.col('100_7').is_in(cesi)).group_by('rok').agg(pl.col('100_7').n_unique()).rename({'100_7':'celkem'}), on='rok', how='left').with_columns((pl.col('skoncilo') / pl.col('celkem')).alias('podil')).sort(by='rok')
podil_koncicich

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1805,2,5,0.4
1807,1,7,0.142857
1810,1,4,0.25
1813,1,6,0.166667
1814,1,9,0.111111
1815,2,10,0.2
1816,1,10,0.1
1818,1,10,0.1
1821,2,21,0.095238


In [17]:
podil_koncicich.filter(pl.col('rok').is_between(1900,2000)).sort(by='podil')

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1903,33,658,0.050152
1918,31,617,0.050243
1954,71,1401,0.050678
1900,26,475,0.054737
1917,32,580,0.055172
1920,64,1145,0.055895
1911,45,804,0.05597
1919,54,957,0.056426
1908,43,749,0.05741


In [91]:
podil_koncicich

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1805,2,5,0.4
1807,1,7,0.142857
1810,1,4,0.25
1813,1,6,0.166667
1814,1,9,0.111111
1815,2,10,0.2
1816,1,10,0.1
1818,1,10,0.1
1821,2,21,0.095238


In [18]:
podil_koncicich.write_parquet(os.path.join('data','podily_konciciho_autorstva_cs.parquet'))

## Nejproduktivnější autorstvo nevydávané po historických zlomech

In [20]:
def nevysli_po(rok, obdobi):
    vysli_pred = df.filter(pl.col("rok").is_between(rok-obdobi,rok)).select(pl.col("100_7")).to_series().to_list()
    vysli_po = set(df.filter(pl.col("rok") > rok).select(pl.col("100_7")).to_series().to_list())
    ukonceni = [v for v in vysli_pred if v not in vysli_po]
    print(f"Filtru vyhovuje {len(ukonceni)} lidí.")
    return df.filter(pl.col("100_7").is_in(ukonceni)).group_by(["100_a","100_7"]).len().sort(by="len",descending=True).head(100)

In [21]:
nevysli_po(1918, 18)

Filtru vyhovuje 5909 lidí.


100_a,100_7,len
str,str,u32
"""Vrchlický, Jaroslav,""","""jk01151037""",257
"""Clifton, Léon""","""xx0111518""",196
"""Vymazal, František,""","""jk01151194""",153
"""Adámek, Karel,""","""jk01010079""",147
"""Lužická, Věnceslava,""","""jk01072706""",116
"""Čech, Svatopluk,""","""jk01021137""",101
"""Arbes, Jakub,""","""jk01010356""",86
"""Tolstoj, Lev Nikolajevič,""","""jn19981002230""",80
"""Sienkiewicz, Henryk,""","""jn19990007946""",75


In [22]:
nevysli_po(1945, 5)

Filtru vyhovuje 2859 lidí.


100_a,100_7,len
str,str,u32
"""Batěk, Alexandr,""","""jk01011106""",322
"""Strong, Pitt,""","""jo2005267810""",321
"""Machar, Josef Svatopluk,""","""jk01072996""",262
"""Svoboda, F. X.,""","""jk01121960""",212
"""Čečetka, František Josef,""","""jk01021086""",166
"""Marešová, Eva,""","""jo2008475985""",154
"""Bass, Eduard,""","""jk01011066""",128
"""Javořická, Vlasta,""","""jk01051344""",122
"""Kapras, Jan,""","""jk01052930""",120


In [23]:
nevysli_po(1989, 25)

Filtru vyhovuje 38556 lidí.


100_a,100_7,len
str,str,u32
"""Dobrovolný, Bohumil,""","""jk01022456""",234
"""Majerová, Marie,""","""jk01073121""",206
"""Seifert, Jaroslav,""","""jk01110657""",196
"""Baudyš, Eduard,""","""jk01011141""",183
"""Říha, Bohumil,""","""jk01110164""",162
"""Hrubín, František,""","""jk01042920""",162
"""Nový, Karel,""","""jk01090808""",128
"""Hostáň, Jan,""","""jk01042385""",114
"""Nor, A. C.,""","""jk01090252""",109


In [24]:
nevysli_po(2000, 9)

Filtru vyhovuje 11806 lidí.


100_a,100_7,len
str,str,u32
"""Kožík, František,""","""jk01062651""",185
"""Foglar, Jaroslav,""","""jk01031485""",129
"""Barner, Gerhard F.""","""jx20040908007""",122
"""Hrabal, Bohumil,""","""jk01042564""",121
"""Marie-Françoise""","""jx20040909019""",114
"""Malíř, František,""","""jk01073272""",98
"""Linc, Vladimír,""","""jk01072129""",98
"""Marek, Jiří,""","""jk01080125""",89
"""Šmahelová, Helena,""","""jk01130497""",85


## Normalizace

Nejdříve porovnáme všechna x-letá období ve 20. století.

In [27]:
%%time
odmlceni = []
for r in range(1967,1970):
    print(r)
    vychazeli_pred = df.filter(pl.col("rok") < r).select(pl.col("100_7")).to_series().to_list()
    vychazeli_po = set(df.filter(pl.col("rok") > (r + 18)).select(pl.col("100_7")).to_series().to_list())
    vychazeli_mezi = df.filter(pl.col("rok").is_between(r, r+18)).select(pl.col("100_7")).to_series().to_list()
    cancellovani = [v for v in vychazeli_pred if (v in vychazeli_po) and (v not in vychazeli_mezi)]
    odmlceni.append({'rok': r, 'podil': len(cancellovani) / len(vychazeli_pred)})

1967
1968
1969
CPU times: total: 53.9 s
Wall time: 48.2 s


In [28]:
pl.DataFrame(odmlceni)

rok,podil
i64,f64
1967,0.011101
1968,0.01261
1969,0.013939


In [29]:
delky_pauzy = [5,10,15,20]
odmlceni = []
for r in range(1820,2020):
    rocni = {'rok': r}
    for delka_pauzy in delky_pauzy:
        print(f"{r} + {delka_pauzy} let")
        vychazeli_pred = set(df.filter(pl.col("rok") < r).select(pl.col("100_7")).to_series().to_list())
        vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).select(pl.col("100_7")).to_series().to_list())
        vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).select(pl.col("100_7")).to_series().to_list())
        cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
        rocni[f'podil_vsech_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
        vychazeli_pred = set(df.filter(pl.col("rok") < r).filter(pl.col('100_7').is_in(cesi)).select(pl.col("100_7")).to_series().to_list())
        vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).filter(pl.col('100_7').is_in(cesi)).select(pl.col("100_7")).to_series().to_list())
        vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).filter(pl.col('100_7').is_in(cesi)).select(pl.col("100_7")).to_series().to_list())
        cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
        rocni[f'podil_cechu_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
    odmlceni.append(rocni)

1820 + 5 let
1820 + 10 let
1820 + 15 let
1820 + 20 let
1821 + 5 let
1821 + 10 let
1821 + 15 let
1821 + 20 let
1822 + 5 let
1822 + 10 let
1822 + 15 let
1822 + 20 let
1823 + 5 let
1823 + 10 let
1823 + 15 let
1823 + 20 let
1824 + 5 let
1824 + 10 let
1824 + 15 let
1824 + 20 let
1825 + 5 let
1825 + 10 let
1825 + 15 let
1825 + 20 let
1826 + 5 let
1826 + 10 let
1826 + 15 let
1826 + 20 let
1827 + 5 let
1827 + 10 let
1827 + 15 let
1827 + 20 let
1828 + 5 let
1828 + 10 let
1828 + 15 let
1828 + 20 let
1829 + 5 let
1829 + 10 let
1829 + 15 let
1829 + 20 let
1830 + 5 let
1830 + 10 let
1830 + 15 let
1830 + 20 let
1831 + 5 let
1831 + 10 let
1831 + 15 let
1831 + 20 let
1832 + 5 let
1832 + 10 let
1832 + 15 let
1832 + 20 let
1833 + 5 let
1833 + 10 let
1833 + 15 let
1833 + 20 let
1834 + 5 let
1834 + 10 let
1834 + 15 let
1834 + 20 let
1835 + 5 let
1835 + 10 let
1835 + 15 let
1835 + 20 let
1836 + 5 let
1836 + 10 let
1836 + 15 let
1836 + 20 let
1837 + 5 let
1837 + 10 let
1837 + 15 let
1837 + 20 let
1838 + 5 l

In [30]:
odmlceni

[{'rok': 1820,
  'podil_vsech_5': 0.14285714285714285,
  'podil_cechu_5': 0.07894736842105263,
  'podil_vsech_10': 0.10204081632653061,
  'podil_cechu_10': 0.05263157894736842,
  'podil_vsech_15': 0.08163265306122448,
  'podil_cechu_15': 0.02631578947368421,
  'podil_vsech_20': 0.061224489795918366,
  'podil_cechu_20': 0.0},
 {'rok': 1821,
  'podil_vsech_5': 0.2,
  'podil_cechu_5': 0.1590909090909091,
  'podil_vsech_10': 0.12727272727272726,
  'podil_cechu_10': 0.09090909090909091,
  'podil_vsech_15': 0.09090909090909091,
  'podil_cechu_15': 0.045454545454545456,
  'podil_vsech_20': 0.07272727272727272,
  'podil_cechu_20': 0.022727272727272728},
 {'rok': 1822,
  'podil_vsech_5': 0.25,
  'podil_cechu_5': 0.23076923076923078,
  'podil_vsech_10': 0.15625,
  'podil_cechu_10': 0.1346153846153846,
  'podil_vsech_15': 0.09375,
  'podil_cechu_15': 0.057692307692307696,
  'podil_vsech_20': 0.0625,
  'podil_cechu_20': 0.038461538461538464},
 {'rok': 1823,
  'podil_vsech_5': 0.22535211267605634,


In [31]:
pl.DataFrame(odmlceni)

rok,podil_vsech_5,podil_cechu_5,podil_vsech_10,podil_cechu_10,podil_vsech_15,podil_cechu_15,podil_vsech_20,podil_cechu_20
i64,f64,f64,f64,f64,f64,f64,f64,f64
1820,0.142857,0.078947,0.102041,0.052632,0.081633,0.026316,0.061224,0.0
1821,0.2,0.159091,0.127273,0.090909,0.090909,0.045455,0.072727,0.022727
1822,0.25,0.230769,0.15625,0.134615,0.09375,0.057692,0.0625,0.038462
1823,0.225352,0.20339,0.126761,0.101695,0.084507,0.050847,0.042254,0.016949
1824,0.185185,0.15942,0.098765,0.072464,0.074074,0.043478,0.037037,0.014493
1825,0.214286,0.2,0.142857,0.114286,0.107143,0.071429,0.035714,0.014286
1826,0.244186,0.236111,0.162791,0.138889,0.116279,0.083333,0.046512,0.027778
1827,0.224719,0.216216,0.146067,0.121622,0.078652,0.067568,0.044944,0.027027
1828,0.265957,0.246753,0.202128,0.168831,0.053191,0.025974,0.053191,0.025974


In [32]:
pl.DataFrame(odmlceni).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime)).write_parquet(os.path.join('data',f'podil_autorstva_s_pauzou.parquet'))

In [33]:
odmlceni = pl.DataFrame(odmlceni).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime))

In [34]:
import altair as alt

In [35]:
odmlceni.filter(pl.col("rok") < datetime.date(year = 2010, month=1, day=1))

rok,podil_vsech_5,podil_cechu_5,podil_vsech_10,podil_cechu_10,podil_vsech_15,podil_cechu_15,podil_vsech_20,podil_cechu_20
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64
1820-01-01 00:00:00,0.142857,0.078947,0.102041,0.052632,0.081633,0.026316,0.061224,0.0
1821-01-01 00:00:00,0.2,0.159091,0.127273,0.090909,0.090909,0.045455,0.072727,0.022727
1822-01-01 00:00:00,0.25,0.230769,0.15625,0.134615,0.09375,0.057692,0.0625,0.038462
1823-01-01 00:00:00,0.225352,0.20339,0.126761,0.101695,0.084507,0.050847,0.042254,0.016949
1824-01-01 00:00:00,0.185185,0.15942,0.098765,0.072464,0.074074,0.043478,0.037037,0.014493
1825-01-01 00:00:00,0.214286,0.2,0.142857,0.114286,0.107143,0.071429,0.035714,0.014286
1826-01-01 00:00:00,0.244186,0.236111,0.162791,0.138889,0.116279,0.083333,0.046512,0.027778
1827-01-01 00:00:00,0.224719,0.216216,0.146067,0.121622,0.078652,0.067568,0.044944,0.027027
1828-01-01 00:00:00,0.265957,0.246753,0.202128,0.168831,0.053191,0.025974,0.053191,0.025974


In [36]:
alt.Chart(odmlceni.filter(pl.col("rok") < datetime.date(year = 2010, month=1, day=1)).to_pandas()).mark_line().encode(alt.Y("podil_cechu_10:Q"),alt.X('rok:T'))

alt.Chart(...)

In [37]:
alt.Chart(odmlceni.filter(pl.col("rok").is_between(datetime.date(year = 1900, month=1, day=1), datetime.date(year = 2000, month=1, day=1))).to_pandas()).mark_line().encode(alt.Y("podil_cechu_15:Q"),alt.X('rok:T'))

alt.Chart(...)

In [38]:
alt.Chart(pl.DataFrame(odmlceni).to_pandas()).mark_line().encode(alt.Y("podil_vsech:Q"),alt.X('rok:T'))

alt.Chart(...)

## Znormalizovaní

In [40]:
pred_normalizaci = set(df.filter(pl.col("rok") <= 1972).select(pl.col("100_7")).to_series().to_list())
po_normalizaci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
behem_normalizace = set(df.filter(pl.col("rok").is_between(1973,1987)).select(pl.col("100_7")).to_series().to_list())

In [41]:
znormalizovani = pred_normalizaci.intersection(po_normalizaci).difference(behem_normalizace)

In [42]:
df.drop_nulls(subset=['700_7','700_a','700_4']).explode(['700_7','700_a','700_4']).filter(pl.col(["700_7"]).is_in(znormalizovani)).filter(pl.col("rok").is_between(1973,1987)).group_by(['700_7','700_a','700_4']).len().sort(by='len',descending=True)

700_7,700_a,700_4,len
str,str,str,u32
"""jk01083128""","""Němeček, Jaroslav,""","""ill""",93
"""jk01031767""","""Franta, Karel,""","""ill""",33
"""jk01101873""","""Rada, Miroslav,""","""ill""",25
"""mzk2004148400""","""Svoboda, Pavel,""","""aut""",13
"""jk01021228""","""Čemus, Otakar,""","""ill""",13
"""jk01052538""","""Kahounová, Jana,""","""aut""",13
"""mzk2007423808""","""Vrba, Kamil,""","""aut""",10
"""jk01070261""","""Kubašta, Vojtěch,""","""ill""",10
"""jk01041100""","""Hísek, Květoslav,""","""ill""",10


In [43]:
df.filter(pl.col("100_7").is_in(znormalizovani) & pl.col("100_7").is_in(cesi)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Lustig, Arnošt,""","""jk01072683""",112
"""Škvorecký, Josef,""","""jk01130413""",108
"""Malíř, František,""","""jk01073272""",98
"""Klaus, Václav,""","""jn19990218045""",92
"""Klíma, Ivan,""","""jk01060513""",81
"""Havel, Václav,""","""jk01040240""",63
"""Šiktanc, Karel,""","""jk01122896""",61
"""Svěrák, Zdeněk,""","""jk01121890""",60
"""Kundera, Milan,""","""jk01070894""",59


In [44]:
df.filter(pl.col("100_7").is_in(znormalizovani)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Lustig, Arnošt,""","""jk01072683""",112
"""Škvorecký, Josef,""","""jk01130413""",108
"""Malíř, František,""","""jk01073272""",98
"""Klaus, Václav,""","""jn19990218045""",92
"""Däniken, Erich von,""","""jn19990001663""",86
"""Klíma, Ivan,""","""jk01060513""",81
"""Havel, Václav,""","""jk01040240""",63
"""Šiktanc, Karel,""","""jk01122896""",61
"""Svěrák, Zdeněk,""","""jk01121890""",60


## Zúnorovaní

In [46]:
pred_unorem = set(df.filter(pl.col("rok") <= 1948).select(pl.col("100_7")).to_series().to_list())
po_revoluci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
v_sedesatkach = set(df.filter(pl.col("rok").is_between(1960,1969)).select(pl.col("100_7")).to_series().to_list())
behem_socialismu = set(df.filter(pl.col("rok").is_between(1949,1987)).select(pl.col("100_7")).to_series().to_list())
behem_padesatek = set(df.filter(pl.col("rok").is_between(1949,1959)).select(pl.col("100_7")).to_series().to_list())

In [47]:
zunorovani = pred_unorem.intersection(po_revoluci).difference(behem_socialismu)
df.filter(pl.col("100_7").is_in(zunorovani)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Tomáš, Eduard,""","""jn99240001194""",28
"""Jerza, J.,""","""jk01051513""",27
"""Listopad, František,""","""jk01072248""",24
"""Rothberg, Gerta""","""jx20041020018""",20
"""Vajtauer, Emanuel,""","""jk01141174""",20
"""MacKhiboney, Will""","""jn19981001796""",20
"""Rotrekl, Zdeněk,""","""jk01102796""",20
"""Jehličková, Božena""","""jo20000080440""",19
"""Seidlová, Anastasie B.""","""jk01110648""",19


In [48]:
df.filter(pl.col("100_7").is_in(zunorovani) & pl.col('100_7').is_in(cesi)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Tomáš, Eduard,""","""jn99240001194""",28
"""Jerza, J.,""","""jk01051513""",27
"""Listopad, František,""","""jk01072248""",24
"""Vajtauer, Emanuel,""","""jk01141174""",20
"""Rotrekl, Zdeněk,""","""jk01102796""",20
"""MacKhiboney, Will""","""jn19981001796""",20
"""Seidlová, Anastasie B.""","""jk01110648""",19
"""Jehličková, Božena""","""jo20000080440""",19
"""Zillich, František""","""jx20050504041""",16


In [49]:
zunorovani2 = pred_unorem.intersection(v_sedesatkach).difference(behem_padesatek)
df.filter(pl.col("100_7").is_in(zunorovani2)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Foglar, Jaroslav,""","""jk01031485""",129
"""Chmelař, František,""","""jk01050240""",107
"""Strejček, Ferdinand,""","""jk01121431""",62
"""Podhradský, Jan,""","""jk01100207""",57
"""Maurois, André,""","""jn19990005492""",53
"""Calma, Marie,""","""jk01020549""",49
"""Polák, Vladimír,""","""jk01100429""",45
"""Simenon, Georges,""","""jn19990007966""",45
"""Zapletal, Štěpán,""","""jk01152123""",43
